In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
from folium.plugins import MarkerCluster
from shapely.geometry import Point
from shapely.ops import unary_union
from branca.element import Template, MacroElement
import numpy as np
from folium import Element
import json
import re

In [ ]:
# 어린이 교통사고 데이터 불러오기
df = pd.read_excel(r"C:\Users\danddi\my_ws\project\mid_project\어린이 교통사고 자료\어린이교통사고전처리완료\all.xlsx")

# 시군구별 위.경도 데이터 불러오기
df_all = pd.read_excel(r"C:\Users\danddi\my_ws\project\mid_project\어린이 교통사고 자료\어린이교통사고전처리완료\total.xlsx")

display(df)
display(df_all)

,Unnamed: 0,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,...,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,어린이보호구역,연도,월,일,시간,도/시,시/구
0,0,2017010200100403,2017년 1월 2일 19시,월요일,서울특별시 중랑구 면목동,차대사람 - 횡단중,안전운전불이행,건조,맑음,단일로 - 기타,...,남,12,경상,0,2017,1,2,19,서울특별시,중랑구
1,1,2017010300100250,2017년 1월 3일 12시,화요일,서울특별시 노원구 하계동,차대차 - 측면충돌,안전운전불이행,건조,맑음,단일로 - 기타,...,남,8,부상신고,0,2017,1,3,12,서울특별시,노원구
2,2,2017010400100297,2017년 1월 4일 15시,수요일,서울특별시 송파구 문정동,차대차 - 측면충돌,안전운전불이행,건조,맑음,단일로 - 기타,...,남,12,중상,0,2017,1,4,15,서울특별시,송파구
3,3,2017010400100442,2017년 1월 4일 19시,수요일,서울특별시 관악구 신림동,차대사람 - 횡단중,안전운전불이행,건조,맑음,교차로 - 교차로부근,...,남,8,경상,0,2017,1,4,19,서울특별시,관악구
4,4,2017010500100479,2017년 1월 5일 21시,목요일,서울특별시 성북구 장위동,차대사람 - 기타,안전운전불이행,건조,맑음,단일로 - 기타,...,남,4,경상,0,2017,1,5,21,서울특별시,성북구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8960,8960,2022122800100308,2022년 12월 28일 14시,수요일,경기도 남양주시 진접읍,차대사람 - 횡단중,신호위반,건조,맑음,교차로 - 교차로안,...,여,7,경상,0,2022,12,28,14,경기도,남양주시
8961,8961,2022122900100255,2022년 12월 29일 13시,목요일,경기도 동두천시 지행동,차대사람 - 횡단중,신호위반,건조,맑음,기타 - 기타,...,남,10,경상,0,2022,12,29,13,경기도,동두천시
8962,8962,2022122900100396,2022년 12월 29일 17시,목요일,경기도 화성시 장지동,차대사람 - 횡단중,기타,건조,맑음,교차로 - 교차로횡단보도내,...,남,7,중상,1,2022,12,29,17,경기도,화성시
8963,8963,2022122900100417,2022년 12월 29일 17시,목요일,경기도 가평군 북면,차대사람 - 기타,안전운전불이행,건조,맑음,기타 - 기타,...,여,10,중상,0,2022,12,29,17,경기도,가평군


,서울특별시 중랑구 면목동,127.081801119,37.586547028
0,서울특별시 노원구 하계동,127.074461,37.636216
1,서울특별시 송파구 문정동,127.125125,37.486870
2,서울특별시 관악구 신림동,126.928349,37.486899
3,서울특별시 성북구 장위동,127.051659,37.615085
4,서울특별시 양천구 신정동,126.840681,37.512113
...,...,...,...
8959,경기도 남양주시 진접읍,127.189899,37.726396
8960,경기도 동두천시 지행동,127.053231,37.890883
8961,경기도 화성시 장지동,127.094583,37.159118
8962,경기도 가평군 북면,127.477707,37.953506


In [ ]:
# 지도 설명 HTML
template = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    top: 5px;
    right: 5px;
    width: 270px;
    height: 220px;
    z-index:9999;
    font-size:14px;
    background: #f0f0f0;
    opacity: 0.9;
    padding: 5px;
    border-radius: 10px;
    ">
    <p>&nbsp;<i class="fas fa-map-marker-alt" style="color:#3C3C8C; font-size:1.7em;"></i>&nbsp;: 마커 (법정동 대표 좌표)</p>
    <p><i class="fas fa-square" style="color:#ff0000; font-size:1.5em;"></i>&nbsp;: 위험도 매우 높음</p>
    <p><i class="fas fa-square" style="color:#AA1EAA; font-size:1.5em;"></i>&nbsp;: 위험도 높음</p>
    <p><i class="fas fa-square" style="color:#40A940; font-size:1.5em;"></i>&nbsp;: 위험도 낮음 </p>
    <p><i class="fas fa-circle" style="color:#FFF064; font-size:1.5em;"></i>&nbsp;: 어린이 보호구역</p>
    <p><i class="fa-regular fa-square" style="color:#646464; font-size:1.5em;"></i>&nbsp;: 법정동 경계</p>

</div>
<div style="
    position: fixed; 
    bottom: 5px;
    left: 5px;
    width: 250px;
    height: 40px;
    z-index:9999;
    font-size:14px;
    background: #f0f0f0;
    opacity: 0.9;
    padding: 5px;
    border-radius: 10px;
    ">
    <p><b>2017년부터 2022년 어린이 교통사고 데이터를 분석하여 위험도를 계산 후 색상으로 표시</p>
</div>
{% endmacro %}
"""

# 지도 정의
new_map = folium.Map(location=(37.53,127), tiles = 'cartodbpositron', zoom_start=10, control_scale=True)

# 마커 클러스터 기능 추가
marker_cluster = MarkerCluster(
    options={
        'showCoverageOnHover': False,
        'zoomToBoundsOnClick': True,}
        ).add_to(new_map)


#행정구역 경계 지도에 추가
gdf_boundary_bjd = gpd.read_file(r"C:\Users\danddi\Downloads\emd.json")
"C:\Users\danddi\my_ws\project\mid_project\folium\emd.json"
display(gdf_boundary_bjd)

gdf_boundary_bjd[gdf_boundary_bjd['EMD_CD'].str[:2]=='11']
gdf_boundary_bjd[gdf_boundary_bjd['EMD_CD'].str[:2]=='41']

seo = gdf_boundary_bjd['EMD_CD'].str[:2]=='11'  # 서울
gye = gdf_boundary_bjd['EMD_CD'].str[:2]=='41'  # 경기
gdf_boundary_bjd = gdf_boundary_bjd[seo | gye]

folium.GeoJson(
    gdf_boundary_bjd,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        # 'color': '#0000ff',
        'color': '#646464',  # 테두리 색상을 하게 설정
        'weight': 1,  # 테두리 두께를 1로 설정
        'fillOpacity': 0,
    }
).add_to(new_map)



# 위험도 계산 함수
def calculate_danger_coeff(df, sg):
    accident_weights = {
        "경상": 0.5, 
        "중상": 1,  
        "부상신고": 0.25, 
        "사망": 2.5 
    }

    accidents_in_location = df[df['시군구'] == sg]
    danger_coeff = 0
    for index, row in accidents_in_location.iterrows():
        accident_info = row['상해정도 정보'].split('<br>')
        for info in accident_info:
            accident_type, num_accidents_of_type = info.split(': ')
            num_accidents_of_type = int(num_accidents_of_type)
            if accident_type in accident_weights:
                danger_coeff += num_accidents_of_type * accident_weights[accident_type]
        # if danger_coeff >= 20:  # 위험도가 20 이상인 경우
            # print(f"시군구: {sg}, 위험도: {danger_coeff}")
    return danger_coeff

# 위험도 기준치 함수
def get_color(danger_coeff):
    if danger_coeff < 7:
        return 'green'
    # elif danger_coeff < 0.01:
    #     return 'yellow'
    elif danger_coeff < 20: 
        return 'purple'
    else:
        return 'red'

# 위험도 계산을 위해 데이터 정리
columns = ["시군구", "경도", "위도"]
df_all.columns = columns
first_all = df_all.groupby('시군구').first().reset_index()

grouped_df = df.groupby(['시군구', '피해운전자 상해정도']).size().reset_index(name='상해정도 갯수')

grouped_df['피해운전자 상해정도'] = grouped_df['피해운전자 상해정도'].astype(str)
grouped_df['상해정도 갯수'] = grouped_df['상해정도 갯수'].astype(str)
grouped_df['상해정도 정보'] = grouped_df[['피해운전자 상해정도', '상해정도 갯수']].apply(lambda x: ': '.join(x), axis=1)

grouped_df_agg = grouped_df.groupby('시군구')['상해정도 정보'].apply(lambda x: '<br>'.join(x)).reset_index()

first_all_merged = first_all.merge(grouped_df_agg, on='시군구', how='left')

geometry = [Point(lon, lat) for lat, lon in zip(first_all_merged['위도'], first_all_merged['경도'])]
gdf_coordinates = gpd.GeoDataFrame(first_all_merged, geometry=geometry, crs='EPSG:4326')
gdf_coordinates = gdf_coordinates[~gdf_coordinates['geometry'].is_empty]

# 위험도 및 마커 지도에 추가
for index, row in gdf_coordinates.iterrows():
    if not row['geometry'].is_empty:
        # 위험도 계산
        danger_coeff = calculate_danger_coeff(first_all_merged, row['시군구'])

        # 위험도에 따른 색상 결정
        color = get_color(danger_coeff)

        # 위험도 소수점 
        danger_coeff = round(danger_coeff, 6)

        # 위험도 등급 구분
        color_kr = {'green': '낮음', 'purple': '높음', 'red': '매우 높음'}[color]

        latitude = row['geometry'].y
        longitude = row['geometry'].x
        # icon = folium.Icon(color=color,icon='info-sign')
        icon = folium.Icon(color=color,icon='person-falling-burst', prefix='fa')
        popup_text = f"시군구: {row['시군구']}<br>{row['상해정도 정보']}<br>위험도: {danger_coeff}<br>위험도 등급: {color_kr}"
        folium.Marker(location=[latitude, longitude], icon=icon, popup=folium.Popup(popup_text, max_width=250)).add_to(marker_cluster)



# 어린이 보호구역 지도에 추가(경기도)
gdf_protect = gpd.read_file(r"C:\Users\danddi\my_ws\project\mid_project\folium\경기 어린이보호구역 위치도.geojson")
def polygon_to_circle(polygon):
    center = polygon.centroid
    distance = center.distance(unary_union(polygon.boundary))
    return center.buffer(distance)

# 좌표계 변환: EPSG:4326 → EPSG:5179 (UTM-K)
gdf_protect = gdf_protect.to_crs(epsg=5179)

# 원 그리기 (반경 300m)
gdf_protect['geometry'] = gdf_protect['geometry'].apply(lambda poly: poly.centroid.buffer(300))

# 다시 EPSG:4326로
gdf_protect = gdf_protect.to_crs(epsg=4326)

# 어린이 보호구역 설정
folium.GeoJson(
    gdf_protect,
    style_function=lambda feature: {
        'fillColor': '#FFF064',
        # 'color': '#FFF064',
        'color': 'transparent',  # 테두리 색상을 투명하게 설정
        'weight': 0.5,  # 테두리 두께
        'fillOpacity': 0.4,
    }
).add_to(new_map)

# 어린이 보호구역 지도에 추가(서울)
with open(r'C:\Users\danddi\my_ws\project\mid_project\folium\서울 어린이보호구역 위치도 수정.json', encoding='utf-8') as f:
    js = json.load(f)

pd_seo_df = pd.DataFrame(js)
# Index(['id', 'fclt_nm', 'gu_nm', 'polstn', 'dong_nm', 'ycrd', 'xcrd', 'mng_no',
#        'addr', 'fclt_se', 'gmtr'],
#       dtype='object')

# gmtr 열에서 x, y 추출
def parse_gmtr(gmtr_str):
    match = re.search(r"MULTIPOINT\(([\d.]+) ([\d.]+)\)", gmtr_str)
    if match:
        return float(match.group(1)), float(match.group(2))
    return None, None

pd_seo_df[['x_tm', 'y_tm']] = pd_seo_df['gmtr'].apply(lambda s: pd.Series(parse_gmtr(s)))

# GeoDataFrame 만들기 (EPSG:5179 → TM좌표계)
gdf_protect_seo = gpd.GeoDataFrame(
    pd_seo_df,
    geometry=[Point(x, y) for x, y in zip(pd_seo_df['x_tm'], pd_seo_df['y_tm'])],
    crs='EPSG:5179'
)

# 반경 300M 원으로 바꾸기 (buffer는 단위가 미터이므로 EPSG:5179에서 적용)
gdf_protect_seo['geometry'] = gdf_protect_seo.geometry.buffer(300)

# EPSG:4326으로 변환 (folium에서 사용 가능하도록)
gdf_protect_seo = gdf_protect_seo.to_crs(epsg=4326)

# GeoJson으로 지도에 추가
folium.GeoJson(
    gdf_protect_seo,
    style_function=lambda feature: {
        'fillColor': '#FFF064',
        'color': 'transparent',
        'weight': 0.5,
        'fillOpacity': 0.4,
    },
).add_to(new_map)


macro = MacroElement()
macro._template = Template(template)
new_map.get_root().add_child(macro)

new_map.save(r'C:\Users\danddi\my_ws\project\mid_project\folium\risk_map.html')



,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,geometry
0,11110101,Cheongun-dong,청운동,"POLYGON ((126.97556 37.58968, 126.97468 37.591..."
1,11110102,Singyo-dong,신교동,"POLYGON ((126.97031 37.58418, 126.97023 37.584..."
2,11110103,Gungjeong-dong,궁정동,"POLYGON ((126.97400 37.58654, 126.97398 37.586..."
3,11110104,Hyoja-dong,효자동,"POLYGON ((126.97356 37.58323, 126.97356 37.583..."
4,11110105,Changseong-dong,창성동,"POLYGON ((126.97353 37.58182, 126.97343 37.581..."
...,...,...,...,...
5060,51830310,Seo-myeon,서면,"POLYGON ((128.40425 38.09607, 128.40503 38.095..."
5061,51830320,Sonyang-myeon,손양면,"MULTIPOLYGON (((128.70421 38.03833, 128.70392 ..."
5062,51830330,Hyeonbuk-myeon,현북면,"MULTIPOLYGON (((128.73582 38.01976, 128.73583 ..."
5063,51830340,Hyeonnam-myeon,현남면,"MULTIPOLYGON (((128.68694 37.89268, 128.68713 ..."
